# Convert The `Food-101` Keras Model to CoreML

This notebook will convert the [Food-101](https://github.com/stratospark/food-101-keras) trained Keras model and classification labels to a CoreML model. Run each script block by pressing `CTRL-Enter` or the `Run` button (Note: _`In [ ]` will change to `In [*]` when the script is running and eventually updating to `In [somenumber]` when done; some might take a while!_).

More information can be found here:
- [Deep learning food classification demo with Keras](http://blog.stratospark.com/deep-learning-applied-food-classification-deep-learning-keras.html)
- Udacity's free [Core ML: Machine Learning of iOS](https://www.udacity.com/course/core-ml--ud1038) course is a great introduction.

## Import dependencies

In [1]:
from keras.models import load_model
import coremltools

## Download `Food-101` Keras model (if needed)

These will be stored in [/tree/models](/tree/models). If you don't want to do this in this notebook but rather when building the docker image, you can download them in the `Dockerfile` by adding these lines after the workspace directory is created:

```
# Download example Food101 Keras model weights and labels
RUN wget -O /workspace/models/keras-food101-model.hdf5 https://s3.amazonaws.com/stratospark/food-101/model4b.10-0.68.hdf5
RUN wget -O /workspace/models/keras-food101-model-labels.txt https://github.com/stratospark/food-101-mobile/raw/43598fdc08500683bbc04f877ae069c38c8ac4c3/model_export/labels.txt
```

In [2]:
# Define the files to download
basePath='/workspace/models/'
files = [
    {'name': 'Food 101 Model file', 'url': 'https://s3.amazonaws.com/stratospark/food-101/model4b.10-0.68.hdf5', 'path': basePath + 'keras-food101-model.hdf5'},
    {'name': 'Food 101 Labels file', 'url': 'https://github.com/stratospark/food-101-mobile/raw/43598fdc08500683bbc04f877ae069c38c8ac4c3/model_export/labels.txt', 'path': basePath + 'keras-food101-model-labels.txt'}
]

# Convenience method that will download the given files (if needed) and show progress
def downloadFiles(files):
    import os.path
    import ipywidgets as widgets
    from urllib.request import urlretrieve
    from IPython.display import display

    progressBar = widgets.FloatProgress(value=0, min=0, max=100.0, step=0.1, description="Progress:")
    
    def dlProgress(count, blockSize, totalSize):
        percentage = min(float(count * blockSize * 100 / totalSize), 100.0)
        progressBar.value = percentage
        
    for file in files:
        path=file['path']
        if not os.path.isfile(path) and not os.access(path, os.R_OK):
            progressBar = widgets.FloatProgress(value=0, min=0, max=100.0, step=0.1, description="Progress:")
            print("Downloading {0}:".format(file['name']))
            display(progressBar)
            urlretrieve(file['url'], path, reporthook=dlProgress)        
        else:
            print("{0} has already been downloaded".format(file['name']))
            
# Download the files (if needed)
downloadFiles(files)

## Load The `Food-101` Keras model

In [3]:
model = load_model(basePath + 'keras-food101-model.hdf5')
class_labels = basePath + 'keras-food101-model-labels.txt'

__List the Food-101 model summary__

In [4]:
model.summary()

## Convert Keras model to CoreML model

In [5]:
coreml_model = coremltools.converters.keras.convert(model, 
                                                    input_names=['image'], 
                                                    output_names=['confidence'], 
                                                    class_labels=class_labels,
                                                    image_input_names='image', 
                                                    image_scale=2./255, 
                                                    red_bias=-1, 
                                                    green_bias=-1, 
                                                    blue_bias=-1)

## Add metadata to the CoreML model

In [6]:
coreml_model.author = 'Jeroen Wesbeek'
coreml_model.license = 'MIT'
coreml_model.short_description = 'Classifies food from an image as one of 101 classes'
coreml_model.input_description['image'] = 'Food image'
coreml_model.output_description['confidence'] = 'Confidence of the food classification'
coreml_model.output_description['classLabel'] = 'Food classification label'

## Inspect the created CoreML model

In [7]:
coreml_model

## Test the CoreML model predictions

__Download a test image and feed it into the CoreML model to get its prediction__

_Note: this will only work on macOS 10.13 or later!_

In [11]:
import requests
from PIL import Image
from io import BytesIO

response = requests.get('https://www.budgetbytes.com/wp-content/uploads/2017/01/Bibimbap-above.jpg')
bibimbap = Image.open(BytesIO(response.content))

In [12]:
bibimbap

In [16]:
import sys
try:
    coreml_model.predict({'image' : bibimbap})
except Exception as inst:
    print("Could not perform model predictions", inst.args)

## Save the CoreML model

_Note: the converted CoreML model will be stored in to [root](http://0.0.0.0:8888/tree/notebook). Please refer to [Apple's documentation](https://developer.apple.com/documentation/coreml) on how to use the CoreML model inside your App._

In [12]:
coreml_model.save('Food101Net.mlmodel')